# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hazard,37.2495,-83.1932,57.58,41,99,6.11,US,1732734080
1,1,Papatowai,-46.5619,169.4708,66.25,52,100,16.02,NZ,1732734082
2,2,Iskateley,67.6803,53.1512,24.26,91,9,9.93,RU,1732734083
3,3,Puerto Natales,-51.7236,-72.4875,64.85,42,0,16.11,CL,1732734122
4,4,Bethel,41.3712,-73.4140,50.18,54,0,9.22,US,1732733841


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     557 non-null    int64  
 1   City        557 non-null    object 
 2   Lat         557 non-null    float64
 3   Lng         557 non-null    float64
 4   Max Temp    557 non-null    float64
 5   Humidity    557 non-null    int64  
 6   Cloudiness  557 non-null    int64  
 7   Wind Speed  557 non-null    float64
 8   Country     553 non-null    object 
 9   Date        557 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 43.6+ KB


In [5]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
city_data_df.dropna()

mask = (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] < max_temp) & (city_data_df["Wind Speed"] < max_wind)
city_data_df = city_data_df.loc[mask].reset_index(drop=True)


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,Raja,8.4596,25.6780,74.59,27,11,3.96,SS,1732734124
1,6,Emerald,-23.5333,148.1667,73.63,83,100,9.17,AU,1732734126
2,22,Chipata,-13.6333,32.6500,77.45,46,96,4.23,ZM,1732734144
3,28,Huarmey,-10.0681,-78.1522,71.85,73,86,9.69,PE,1732734007
4,34,Pundong,-7.9522,110.3486,78.37,86,100,5.53,ID,1732734157
...,...,...,...,...,...,...,...,...,...,...
78,546,Kasungu,-13.0333,33.4833,75.72,46,36,3.87,MW,1732734764
79,548,Mersing,2.4312,103.8405,75.96,95,100,3.96,MY,1732734766
80,552,Bria,6.5423,21.9863,74.89,43,5,2.66,CF,1732734770
81,555,Las Palmas,28.4204,-14.0131,72.39,53,0,2.30,ES,1732734774


In [6]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     83 non-null     int64  
 1   City        83 non-null     object 
 2   Lat         83 non-null     float64
 3   Lng         83 non-null     float64
 4   Max Temp    83 non-null     float64
 5   Humidity    83 non-null     int64  
 6   Cloudiness  83 non-null     int64  
 7   Wind Speed  83 non-null     float64
 8   Country     83 non-null     object 
 9   Date        83 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 6.6+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Raja,SS,8.4596,25.6780,27,None
1,Emerald,AU,-23.5333,148.1667,83,None
2,Chipata,ZM,-13.6333,32.6500,46,None
3,Huarmey,PE,-10.0681,-78.1522,73,None
4,Pundong,ID,-7.9522,110.3486,86,None
...,...,...,...,...,...,...
78,Kasungu,MW,-13.0333,33.4833,46,None
79,Mersing,MY,2.4312,103.8405,95,None
80,Bria,CF,6.5423,21.9863,43,None
81,Las Palmas,ES,28.4204,-14.0131,53,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000  # search radius in meters (10 km)
categories = "accommodation.hotel"  # Geoapify category for hotels
limit = 5  # Return only the nearest 5 hotels
geoapify_key = "566ca9c640234d4e808989cd03e341f5"  # Your Geoapify API key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Check if the API key is correct
print(f"Using Geoapify API Key: {geoapify_key}")  # Debugging line, can be removed later

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Update the filters and bias with the correct lat/lng
    filters = f"circle:{lng},{lat},{radius}"  # Correct filter for search radius
    bias = f"proximity:{lng},{lat}"  # Correct bias for proximity

    # Set the params dictionary
    params = {
        "categories": categories,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key
    }

    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)

        # Check if the response is successful (status code 200)
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for {row['City']}")
            hotel_df.loc[index, "Hotel Name"] = "Error in API request"
            continue
        
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and len(name_address["features"]) > 0:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except (KeyError, IndexError, requests.exceptions.RequestException) as e:
        # Handle errors such as missing data or API request failure
        hotel_df.loc[index, "Hotel Name"] = "Error in API request"
        print(f"Error fetching hotel for {row['City']}: {e}")

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the DataFrame to check results
hotel_df.head()

Starting hotel search
Using Geoapify API Key: 566ca9c640234d4e808989cd03e341f5
Raja - nearest hotel: No hotel found
Emerald - nearest hotel: Route 66 Motor Inn
Chipata - nearest hotel: Kamocho Gueshouse
Huarmey - nearest hotel: Hostal Santa Rosa
Pundong - nearest hotel: Puri Ganjuran
Upington - nearest hotel: Die Eiland Holiday Ressort
Hawaiian Paradise Park - nearest hotel: No hotel found
Dampit - nearest hotel: No hotel found
Kaga-Bandoro - nearest hotel: No hotel found
Alice Springs - nearest hotel: Aurora Alice Springs
Santa Elena de Uairen - nearest hotel: Posada Villa Fairmont
Abra Pampa - nearest hotel: Residencia El Norte
San Julián - nearest hotel: No hotel found
Molina - nearest hotel: La Cabaña
Turbat - nearest hotel: No hotel found
Jiwani - nearest hotel: No hotel found
Tanguiéta - nearest hotel: Hôtel Atacora
Sola - nearest hotel: No hotel found
Shakawe - nearest hotel: EdenWood guesthouse
Mzimba - nearest hotel: No hotel found
Goz Beida - nearest hotel: No hotel found
Ava

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Raja,SS,8.4596,25.6780,27,No hotel found
1,Emerald,AU,-23.5333,148.1667,83,Route 66 Motor Inn
2,Chipata,ZM,-13.6333,32.6500,46,Kamocho Gueshouse
3,Huarmey,PE,-10.0681,-78.1522,73,Hostal Santa Rosa
4,Pundong,ID,-7.9522,110.3486,86,Puri Ganjuran


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",          
    "Lat",          
    geo=True,       
    tiles="EsriNatGeo",  # Choose the tile layer
    frame_width=700,
    frame_height=500,
    color="City",       # You can customize the color based on the city name or any other variable
    hover_cols=["City", "Country", "Hotel Name"]  # Display these columns in the hover popup
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)